# AI Tools for Actuaries
## Chapter 3: Poisson GLM in Python
### Author: Michael Mayer and Mario Wuthrich
### Version April 2025

In [1]:
# Import required libraries
import time

import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import glm

### Define Poisson Deviance Loss Function
We scale with $10^2$ for better visibility


In [2]:
def poisson_deviance(pred, obs):
    """Average Poisson deviance for claim counts."""
    dev_sum = 2 * (np.sum(pred) - np.sum(obs) + np.sum(np.log((obs / pred) ** obs)))
    return 100 * dev_sum / len(pred)

### Load Data

In [3]:
df = pd.read_parquet("../../Data/freMTPL2freq.parquet")
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 678007 entries, 0 to 678006
Data columns (total 14 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   IDpol       678007 non-null  float64 
 1   Exposure    678007 non-null  float64 
 2   Area        678007 non-null  category
 3   VehPower    678007 non-null  int32   
 4   VehAge      678007 non-null  int32   
 5   DrivAge     678007 non-null  int32   
 6   BonusMalus  678007 non-null  int32   
 7   VehBrand    678007 non-null  category
 8   VehGas      678007 non-null  category
 9   Density     678007 non-null  int32   
 10  Region      678007 non-null  category
 11  ClaimTotal  678007 non-null  float64 
 12  ClaimNb     678007 non-null  float64 
 13  LearnTest   678007 non-null  object  
dtypes: category(4), float64(4), int32(5), object(1)
memory usage: 41.4+ MB
None


,IDpol,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,Region,ClaimTotal,ClaimNb,LearnTest
0,1.0,0.10,D,5,0,55,50,B12,Regular,1217,R82,0.0,0.0,L
1,3.0,0.77,D,5,0,55,50,B12,Regular,1217,R82,0.0,0.0,L
2,5.0,0.75,B,6,2,52,50,B12,Diesel,54,R22,0.0,0.0,L
3,10.0,0.09,B,7,0,46,50,B12,Diesel,76,R72,0.0,0.0,L
4,11.0,0.84,B,7,0,46,50,B12,Diesel,76,R72,0.0,0.0,L


### Preprocess Data for GLM

In [4]:
# Create GLM features
driv_age_lev = ["18-20", "21-25", "26-30", "31-40", "41-50", "51-70", "71+"]
driv_age_lev2 = ["31-40", *[lev for lev in driv_age_lev if lev != "31-40"]]

df = df.assign(
    AreaGLM=lambda x: x.Area.cat.codes + 1,  # Make Area code continuous
    VehPowerGLM=lambda x: pd.Categorical(x.VehPower.clip(0, 9)),  # Make categorical
    # Create age categories
    VehAgeGLM=lambda x: pd.cut(
        x.VehAge, bins=[-1, 5, 12, 101], labels=["0-5", "6-12", "12+"]
    ),
    DrivAgeGLM=lambda x: pd.Categorical(
        pd.cut(x.DrivAge, bins=[17, 20, 25, 30, 40, 50, 70, 101], labels=driv_age_lev),
        categories=driv_age_lev2,
    ),
    BonusMalusGLM=lambda x: x.BonusMalus.clip(0, 150),  # Censor bonus-malus level
    DensityGLM=lambda x: np.log(x.Density),  # Log-transform density
)

### Split into Learning and Test Sets

In [5]:
# Split data (this uses the same split and order as in Wuthrich-Merz, Springer 2023)
learn, test = (df[df.LearnTest == subset].copy() for subset in ("L", "T"))

print(f"Learning set size: {learn.shape[0]}")
print(f"Test set size: {test.shape[0]}")

Learning set size: 610206
Test set size: 67801


### GLM Analysis

In [6]:
# Fit GLM
start_time = time.time()

model = glm(
    "ClaimNb ~ DrivAgeGLM + VehBrand + VehGas + DensityGLM + AreaGLM",
    data=learn,
    offset=np.log(learn["Exposure"]),
    family=sm.families.Poisson(),
)

glm_results = model.fit()
print(f"Time taken: {time.time() - start_time:.2f} seconds\n")

# Display model summary
print(glm_results.summary())

Time taken: 2.19 seconds

                 Generalized Linear Model Regression Results                  
Dep. Variable:                ClaimNb   No. Observations:               610206
Model:                            GLM   Df Residuals:                   610186
Model Family:                 Poisson   Df Model:                           19
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -98513.
Date:                Mon, 14 Apr 2025   Deviance:                   1.5138e+05
Time:                        10:38:35   Pearson chi2:                 1.02e+06
No. Iterations:                     7   Pseudo R-squ. (CS):           0.004052
Covariance Type:            nonrobust                                         
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercep

### Calculate Deviance Losses

In [7]:
# Get predictions
learn["GLM"] = glm_results.predict(learn) * learn["Exposure"]
test["GLM"] = glm_results.predict(test) * test["Exposure"]

# Calculate in-sample and out-of-sample deviance
learn_deviance, test_deviance = (
    poisson_deviance(x.GLM, x.ClaimNb) for x in (learn, test)
)

print("Deviance Losses:")
print(f"Learning sample: {learn_deviance:.3f}")
print(f"Test sample: {test_deviance:.3f}")

Deviance Losses:
Learning sample: 24.807
Test sample: 25.027
